In [ ]:
%matplotlib inline

In [ ]:
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.integrate as integrate
from scipy.special import fresnel

# PHYS 395 - week 2

**Matt Wiens - #301294492**

This notebook will be organized similarly to the lab script, with major headings corresponding to the headings on the lab script.

*The TA's name (Ignacio) will be shortened to "IC" whenever used.*

## Setup

In [ ]:
# Set default plot size
plt.rcParams["figure.figsize"] = (10, 7)

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

# Session 1

# Numerical error homework

## 1. Truncation error

Truncation error is the error involved in approximating an infinite sum by a finite sum. For example, the truncation error involved in approximating $e^x$ to the first three terms of its Taylor series is

\begin{equation}
    \left| \, e^x - \left(1 + x + \frac{x^2}{2}\right) \, \right|
    .
\end{equation}

For numerical integration, truncation error comes in to play because we are approximating an integral (an infinite sum) by a finite sum.

## 2. Rounding error

Given any algorithm which produces a result, rounding error is the difference between producing that result using exact arithmetic and producing the result using finite-precision, rounded arithmetic (floating point arithmetic). On a computer, a number called "machine epsilon" gives you the upper bound of the relative error that occurs in floating point arithmetic due to rounding.

## 3. Demonstrating floating point arithmetic error

In [ ]:
print(7 / 3 - 4 / 3 - 1)

The above in exact arithmetic should be $0$. But due to error in floating point arithmetic it is instead a non-zero small number.

# Numerical integration 

Our goal in this section will be to investigate how we can approximate integrals according to different "rules".    

## Simple rule

Consider some function $f$ defined on an interval $[a, b]$. A simple rule to approximate the integral of $f$ over this interval is to take $N + 1$ evenly spaced points along the interval $\{x_1, x_2, \ldots, x_{N + 1}\}$ and then add up the area of the rectangles induced by these points (see the picture in the lab script). Note that we always have $x_1 = a$ and $x_N = b$. This gives us the following approximation:

\begin{equation}
    \int_a^b f(x) dx \approx \sum_{i = 1}^N f(x_i) h
    ,
\end{equation}

where $h$ is the width of each rectangle given by

\begin{equation}
    h = \frac{b - a}{N}
    .
\end{equation}

In [ ]:
def integrate_left_reimann(y: np.ndarray, h: float) -> float:
    """Approximates an integral using the left Reimann sum."""
    return np.sum(y * h)

Now we will approximate the integral of $\sin(x)$ over $[0, \frac{\pi}{2}]$ using logarithmically spaced values of $N$. We will plot the absolute error as a function of the bin width $h$.

In [ ]:
# Generate a number of N values and the corresponding h values
num_ns = 50

# We need to be a little careful since we want the N values to
# be logarithmically space, but we also require that each N is an integer.
n_vals = np.round(np.logspace(1, 6, num_ns)).astype(int)
h_vals = np.pi / (2 * n_vals)

Before we plot the absolute errors, note that

\begin{equation}
    \int_0^{\frac{\pi}{2}} \sin(x) dx = 1
    .
\end{equation}

In [ ]:
# For each N value calculate the absolute error
errors = np.zeros(num_ns)

for idx, n in enumerate(n_vals):
    xs = np.linspace(0, np.pi / 2, n)
    ys = np.sin(xs)

    errors[idx] = abs(1 - integrate_left_reimann(ys, np.pi / (2 * n)))

In [ ]:
# Make a scatter plot
_, ax = plt.subplots()

plt.loglog(h_vals, errors, "o")

# Labels
ax.set_xlabel(r"$h$")
ax.set_ylabel("abs error");

Here we see a linear relationship in the log-log plot. This means that the error E is related to $h$ through some relationship of the form

\begin{equation}
    E = A h^\alpha
    .
\end{equation}

This is because a linear relationship in log-log can be expressed as

\begin{align}
    &\log E = \alpha \log h + \log A \\
    &\iff \log E = \log \left( A h^\alpha \right) \\
    &\iff E = A h^\alpha
    .
\end{align}

By inspection, I would guess that $\alpha = 1$ and $A = 1$.

Now we will try to fit the log values we obtained to a line.

In [ ]:
# Find the linear coefficients for the log-log relationship
coeffs = np.polyfit(x=np.log(h_vals), y=np.log(errors), deg=1)

# Translate to A and h
print("A = %.2f" % np.exp(coeffs[1]))
print("alpha = %.2f" % coeffs[0])

My guess for $\alpha$ was correct, but the value for $A$ I guessed was not quite right (it's difficult to make out on the above plot what $A$ should be with any precision).

Using the coefficients we calculated, we can estimate that to get an error less than or equal to $10^{-8}$ we would need a width of 

\begin{align}
    h &= \left( \frac{E}{A} \right)^{\frac{1}{\alpha}} \\
      &\approx \frac{10^{-8}}{0.14} \\
      &\approx 8 \cdot 10^{-8}
      ,
\end{align}

which means we need at least $20916033$ slices using this method. Note that the size of $h$ depends linearly on the error $E$ in this case.

In [ ]:
# Supporting calculations for above text block
needed_h = (1e-8 / np.exp(coeffs[1])) ** (1 / coeffs[0])
needed_n = np.pi / 2 / needed_h

print("approx h required: %e" % needed_h)
print("approx N required: %f" % needed_n)

## Trapezoid rule

On the lab script we are shown a picture of how the trapezoid rule works. Given $x$ values $\{x_0, x_1, x_2, x_3\}$ and corresponding function values $\{y_0, y_1, y_2, y_3\}$. There are many ways to compute the sum of the trapezoids: for each trapezoid I will add the the rectangle induced the height of the right endpoint together with the remaining half rectangle. This gives us the formula for the sum $S$ as

\begin{align}
    S &= \sum_{i = 1}^3
        \left(
            y_i h
            + \frac{1}{2} \left( y_{i - 1} - y_i \right) h
        \right) \\
      &= \frac{h}{2} \sum_{i = 1}^3
        \left(
            y_{i - 1} + y_i
        \right)
\end{align}

where $h = x_i - x_{i - 1}$ (assumed constant).

Now, using the trapezoid rule, we will repeat the investigation we performed when using the left Riemann approximation. We will keep the same number of $N$ values to test and the corresponding $h$ values we used previously.

In [ ]:
# For each N value calculate the absolute error
errors = np.zeros(num_ns)

for idx, n in enumerate(n_vals):
    xs = np.linspace(0, np.pi / 2, n + 1)
    ys = np.sin(xs)

    errors[idx] = abs(1 - integrate.trapz(y=ys, dx=np.pi / (2 * n)))

In [ ]:
# Make a scatter plot
_, ax = plt.subplots()

plt.loglog(h_vals, errors, "o")

# Labels
ax.set_xlabel(r"$h$")
ax.set_ylabel("abs error");

Here we can see that the log-log relationship is still linear. However, for the same values of $h$ the error in the trapezoidal rule is much lower.

We'll calculate the $A$ and $\alpha$ parameters and estimate what $h$ and $N$ we would need to keep the absolute error below $10^{-8}$.

In [ ]:
# Find the linear coefficients for the log-log relationship
coeffs = np.polyfit(x=np.log(h_vals), y=np.log(errors), deg=1)

# Translate to A and h
print("A = %.2f" % np.exp(coeffs[1]))
print("alpha = %.2f" % coeffs[0])

The dependence of $h$ on the error $E$ is now squared.

In [ ]:
# What is the greatest h/ least N we can use?
needed_h = (1e-8 / np.exp(coeffs[1])) ** (1 / coeffs[0])
needed_n = np.pi / 2 / needed_h

print("approx h required: %e" % needed_h)
print("approx N required: %f" % needed_n)

To compare with the left Riemann approximation, we need far fewer steps!

## Simpson's method

Note that for Simpson's method we need an *even* number of slices!

First we'll plot the errors for different values of $h$. Note that we need to be careful to make sure $N$ is even here, so we'll adjust each $N$ value so that this holds.

In [ ]:
# We need to be a little careful since we want the N values to
# be logarithmically space, but we also require that each N is an integer.
simps_n_vals = n_vals + n_vals % 2
simps_h_vals = np.pi / (2 * n_vals)

In [ ]:
# For each N value calculate the absolute error
errors = np.zeros(num_ns)

for idx, n in enumerate(simps_n_vals):
    xs = np.linspace(0, np.pi / 2, n + 1)
    ys = np.sin(xs)

    errors[idx] = abs(1 - integrate.simps(y=ys, dx=np.pi / (2 * n)))

In [ ]:
# Make a scatter plot
_, ax = plt.subplots()

plt.loglog(simps_h_vals, errors, "o")

# Labels
ax.set_xlabel(r"$h$")
ax.set_ylabel("abs error");

Here we can clearly see that Simpson's method outperforms both methods we looked at before. We quickly reach the limits of machine precision as we increase $N$. The dependence is still linear, although due to the limits of machine precision, we cannot show the full relationship we would get with exact arithmetic here.

If wanted to fit the linear relationship we need to look at the values of $N$ that lead to $h \approx 10^{-3}$ and higher.

In [ ]:
# Last h value index to keep
last_h_idx = 20

# Find the linear coefficients for the log-log relationship
coeffs = np.polyfit(
    x=np.log(simps_h_vals[: last_h_idx + 1]), y=np.log(errors[: last_h_idx + 1]), deg=1
)

# Translate to A and h
print("A = %.2f" % np.exp(coeffs[1]))
print("alpha = %.2f" % coeffs[0])

The order of dependence of $h$ on $E$ is to the fourth power. Now we can again determine what is the greatest $h$/least $N$ we need to keep the error under $10^{-8}$.

In [ ]:
# What is the greatest h/ least N we can use?
needed_h = (1e-8 / np.exp(coeffs[1])) ** (1 / coeffs[0])
needed_n = np.pi / 2 / needed_h

print("approx h required: %e" % needed_h)
print("approx N required: %f" % needed_n)

As can be seen from the $\alpha$ value or by looking that the greatest $N$ required, Simpson's method is about $100$ times as efficient as the trapezoid rule and far, far more efficient as the left Riemann sum approximation for this integral.

## Timing comparison

Let's compare how long each of these methods take. While we know Simpson's method uses fewer steps, for example, each step could take longer, so up to now, we still don't know which method is "best". We'll now time each method using the $N$ value for each method which gives error below $10^{-8}$.

In [ ]:
# N values
test_n_vals = n_vals + n_vals % 2
test_h_vals = np.pi / (2 * n_vals)

# Left Riemann
n = 20916033
xs = np.linspace(0, np.pi / 2, n + 1)
ys = np.sin(xs)
h = np.pi / (2 * n)

start = timer()

integrate_left_reimann(y=ys, h=h)

lriem_time = timer() - start

# Trapezoid
n = 4535
xs = np.linspace(0, np.pi / 2, n + 1)
ys = np.sin(xs)
h = np.pi / (2 * n)

start = timer()

integrate.simps(y=ys, dx=h)

trapz_time = timer() - start

# Simpson's
n = 43
xs = np.linspace(0, np.pi / 2, n + 1)
ys = np.sin(xs)
h = np.pi / (2 * n)

start = timer()

integrate.simps(y=ys, dx=h)

simps_time = timer() - start

In [ ]:
# Print the times
print("Left Riemann:\t%.5f" % lriem_time)
print("Trapezoid:\t%.5f" % trapz_time)
print("Simpson's:\t%.5f" % simps_time)

At least for this integral, Simpson's method is by bar the fastest.

# Session 1 homework


## The Fresnel integral

The Fresnel integral is given by the following function:

\begin{equation}
    C(x) = \int_0^x \cos \left( \frac{\pi}{2} x^2 \right) dx
    .
\end{equation}

Let's first plot the integrand on the range $[0, 7.1]$.    

In [ ]:
xs = np.linspace(0, 7.1, 500)
ys = np.cos(np.pi / 2 * xs ** 2)

# Plot
_, ax = plt.subplots()

plt.plot(xs, ys)

ax.set_xlabel(r"$x$");

Using SciPy, we can evaluate $C$ at $x = 7.1$.

In [ ]:
_, c_val = fresnel(7.1)

print(c_val)

### Using Simpson's method

If we used the numerical integration methods we discussed above, however, we would need a very large number of slices $N$. This is because, as we can see in the above plot, the integrand of the Fresnel function changes rapidly, which means that any approximation per slice, especially as $x$ gets large, is going to be bad. In this case, it might make more sense to have fewer slices near the beginning of the interval (when the integrand doesn't change as rapidly), and more slices as we progress through the interval, since the integrand is changing more and more with increasing $x$.

Let's find what $h$ and $N$ we need using Simpson's method to keep the error less then $10^{-8}$.

In [ ]:
# For each N value calculate the absolute error
simps_n_vals = n_vals + n_vals % 2
simps_h_vals = 7.1 / simps_n_vals

errors = np.zeros(num_ns)

for idx, n in enumerate(simps_n_vals):
    xs = np.linspace(0, 7.1, n + 1)
    ys = np.cos(np.pi / 2 * xs ** 2)

    errors[idx] = abs(c_val - integrate.simps(y=ys, dx=simps_h_vals[idx]))

In [ ]:
# Plot the errors
_, ax = plt.subplots()

plt.loglog(simps_h_vals, errors, "o")

# Labels
ax.set_xlabel(r"$h$")
ax.set_ylabel("abs error");

We'll determine the coefficients of the log-log relationship using the $h$ values that appear linear on the above plot.

In [ ]:
# Choose which h indices we'll use
first_h_idx = 8
last_h_idx = 35

# Find the linear coefficients for the log-log relationship
coeffs = np.polyfit(
    x=np.log(simps_h_vals[first_h_idx : last_h_idx + 1]),
    y=np.log(errors[first_h_idx : last_h_idx + 1]),
    deg=1,
)

# Translate to A and h
print("A = %.2f" % np.exp(coeffs[1]))
print("alpha = %.2f" % coeffs[0])

Note that determining these coefficients is more sensitive to the data points chosen compared to the cases we looked at earlier. Nevertheless, the coefficients are relatively stable at the above values.

In [ ]:
# What is the greatest h/ least N we can use?
needed_h = (1e-8 / np.exp(coeffs[1])) ** (1 / coeffs[0])
needed_n = 7.1 / needed_h

print("approx h required: %e" % needed_h)
print("approx N required: %f" % needed_n)

The number of steps $N$ is much higher than what we needed for the simpler integral we first looked at.

### Using non-uniformly spaces points

Let's determine what $N$ we need using SciPy's implementation of fixed-order Gaussian quadrature.

In [ ]:
# The integrand
integrand = lambda x: np.cos(np.pi / 2 * x ** 2)

In [ ]:
# Calculate the errors. We'll use fewer N values here because it can
# take quite awhile using the N values we've been using so far.
quad_num_ns = 50
quad_n_vals = np.round(np.logspace(1, 3.5, quad_num_ns)).astype(int)
errors = np.zeros(quad_num_ns)

for idx, n in enumerate(quad_n_vals):
    errors[idx] = abs(c_val - integrate.fixed_quad(func=integrand, a=0, b=7.1, n=n)[0])

In [ ]:
# Plot the errors against N
_, ax = plt.subplots()

plt.loglog(quad_n_vals, errors, "o")

# Labels
ax.set_xlabel(r"$N$")
ax.set_ylabel("abs error");

By inspecting the above plot, it looks like we can use around $N \approx 45$ to reach an absolute error below $10^{-8}$. Let's verify this.

In [ ]:
# Error with N = 90
this_error = abs(c_val - integrate.fixed_quad(func=integrand, a=0, b=7.1, n=45)[0])

print("error with N=%d: %e" % (45, this_error))

SciPy also provides another function which lets us specify the tolerance specifically. Let's try it with a tolerance of $10^{-8}$.

In [ ]:
print(
    "value:\t%e\nerror:\t%e"
    % integrate.quadrature(func=integrand, a=0, b=7.1, tol=1e-8)
)

## Integrating improper integrals

We can also use a SciPy quadrature function to evaluate improper integrals like

\begin{equation}
    \int_{-\infty}^\infty \exp \left(- \frac{x^2}{2} \right) dx = \sqrt{2 \pi}
    .
\end{equation}

In [ ]:
print(
    "value:\t%e\nerror:\t%e"
    % integrate.quad(func=lambda x: np.exp(-(x ** 2) / 2), a=-np.inf, b=np.inf)
)

## Monte Carlo integration

Here we'll revisit the Fresnel integral but this time using Monte Carlo integration. We'll keep using the $N$ values in our `n_vals` array; note however that the interpretation behind $N$ is different: before $N$ was the number of slices and now it is the sample size.

In [ ]:
# Calculate errors
errors = np.zeros(num_ns)

for idx, n in enumerate(n_vals):
    xs = np.random.uniform(0, 7.1, n)
    ys = np.cos(np.pi / 2 * xs ** 2)

    est = 7.1 * np.sum(ys) / n

    errors[idx] = abs(c_val - est)

In [ ]:
# Plot the errors against N
_, ax = plt.subplots()

plt.loglog(n_vals, errors, "o")

# Labels
ax.set_xlabel(r"$N$")
ax.set_ylabel("abs error");

For estimating the Fresnal integral, Monte Carlo seems like a poor choice compared to the other options we've investigated.

# Session 2

# Numerical differentiation

We can approximate a derivative of a function $f$ numerically with

\begin{equation}
    \frac{df}{dx} \approx \frac{f(x + h) - f(x)}{h}
    ,
\end{equation}

where $h$ is a finite number.

Let's estimate the derivative of $f(x) = x^{0.8}$ evaluated at $x = 1$ using different values of $h$. Note that

\begin{equation}
    f^\prime(1) = 0.8
    .
\end{equation}

In [ ]:
# The function and where we're going to evaluate its derivative
f = lambda x: x ** 0.8
eval_pt = 1
exact_val = 0.8

Let's plot the function and its derivative quickly before trying numerical differentiation.

In [ ]:
# Range to plot over
xs = np.linspace(0.1, 2, 500)

# Plot
_, ax = plt.subplots()

plt.plot(xs, f(xs))
plt.plot(xs, 0.8 / xs ** 0.2)

# Legend and axis label
ax.legend([r"$f$", r"$f^\prime$"])
ax.set_xlabel(r"$x$");

The derivative is smooth and doesn't change much, so I don't anticipate any problems with numerical differentiation.

Now let's try to approximate the derivative

In [ ]:
# The h values to use
h_vals = 10 ** np.arange(-15, 0).astype(float)
num_hs = len(h_vals)

In [ ]:
# Calculate the errors
errors = np.zeros(num_hs)

for idx, h in enumerate(h_vals):
    est = (f(eval_pt + h) - f(eval_pt)) / h
    errors[idx] = abs(est - exact_val)

In [ ]:
# Plot the errors
_, ax = plt.subplots()

plt.loglog(h_vals, errors, "o")

# Labels
ax.set_xlabel(r"$h$")
ax.set_ylabel("abs error");

This is at first an unintuitive result: after a certain point as $h$ gets smaller, the approximation gets worse. This is due to limits in finite-precision arithmetic.

Let's try to come up with a better approximation to the derivative using Taylor expansions. Using

\begin{equation}
    f(x \pm h) = f(x) \pm h f^\prime(x) + \mathcal{O}(h^2)
    ,
\end{equation}

we have

\begin{align}
    f(x + h) - f(x - h)
        &=
           \left(
            f(x) + h f^\prime(x) + \mathcal{O}(h^2)
           \right)
           -
           \left(
            f(x) - h f^\prime(x) + \mathcal{O}(h^2)
           \right)
        \\
        &= 2 h f^\prime(x) + \mathcal{O}(h^2)
        ,
\end{align}

or

\begin{equation}
    \frac{f(x + h) - f(x - h)}{2 h} = f^\prime(x) + \mathcal{O}(h^2)
    .
\end{equation}

(Note that the remaining terms are actually $\mathcal{O}(h^3)$ which you can see if you derive the expression using more terms in the Taylor expansions.)

Is this actually better? Let's investigate.

In [ ]:
# Calculate the errors
errors = np.zeros(num_hs)

for idx, h in enumerate(h_vals):
    est = (f(eval_pt + h) - f(eval_pt - h)) / (2 * h)
    errors[idx] = abs(est - exact_val)

In [ ]:
# Plot the errors
_, ax = plt.subplots()

plt.loglog(h_vals, errors, "o")

# Labels
ax.set_xlabel(r"$h$")
ax.set_ylabel("abs error");

This is quite a bit better, if we make sure to take $h$ large enough that we don't bring in too much error from finite-precision arithmetic.

# Solutions to ordinary differential equations

## Euler method

Consider a ball which starts at a height of $1$m and with initial velocity $5$ m/s. The exact trajectory $y(t)$ of this ball is given by

\begin{equation}
    y(t) = -\frac{g}{2} t^2 + 5 t
    .
\end{equation}

Let's plot this trajectory.

In [ ]:
# Gravity constant
g = 9.81

# Plot trajectory
_, ax = plt.subplots()

ts_exact = np.linspace(0, 1, 500)
ys_exact = -g / 2 * ts_exact ** 2 + 5 * ts_exact + 1

plt.plot(ts_exact, ys_exact)

ax.set_xlabel(r"$t$")
ax.set_ylabel(r"$y$");

Now let's approximate the trajectory using Euler's method.

In [ ]:
# Time step
h = 0.01
num_steps = int(1 / h) + 1

# Set up arrays for position and velocity
ts = np.arange(0, 1 + h, step=h)
ys = np.zeros(num_steps)
vs = np.zeros(num_steps)

# Record initial velocity and position
ys[0] = 1
vs[0] = 5

In [ ]:
# Apply Euler's method
for i in range(1, num_steps):
    vs[i] = vs[i - 1] - h * g
    ys[i] = ys[i - 1] + h * vs[i - 1] 

Now let's plot the approximated trajectory against the exact trajectory.

In [ ]:
# Plot
_, ax = plt.subplots()

plt.plot(ts_exact, ys_exact)
plt.plot(ts, ys, "o-")

# Labels and legend
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"$y$")

ax.legend(["exact", "Euler"]);

The agreement is surprisingly good. However, we see that at later times the approximation gets worse; this is due to accumulation of error that occurs in Euler's method.

## Better methods

Now let's use SciPy's `solve_ivp` function to solve the free fall problem.

In [ ]:
res = integrate.solve_ivp(fun=lambda t, y: [y[1], -g], t_span=(0, 1), y0=[1, 5])

Let's plot this alongside the exact trajectory.

In [ ]:
# Plot
_, ax = plt.subplots()

plt.plot(ts_exact, ys_exact)
plt.plot(res.t, res.y[0], "o-")

# Labels and legend
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"$y$")

ax.legend(["exact", "ivp_solve"]);

Very few time points here but also extremely accurate. Let's use more time points, using the same `ts` array we used for Euler's method.

In [ ]:
# Compute
res = integrate.solve_ivp(
    fun=lambda t, y: [y[1], -g], t_span=(0, 1), y0=[1, 5], t_eval=ts
)

# Plot
_, ax = plt.subplots()

plt.plot(ts_exact, ys_exact)
plt.plot(res.t, res.y[0], "o-")

# Labels and legend
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"$y$")

ax.legend(["exact", "ivp_solve"]);

Compare this plot with Euler's method above. Both used the same number of time points, but in terms of accuracy `ivp_solve` performed much better.

## Applications

Consider the equation of motion for a damped harmonic oscillator with forcing

\begin{equation}
    \frac{d^2 x}{dt^2} + 2 \gamma \omega_0 \frac{dx}{dt} + \omega_0^2 x = f
\end{equation}

(see the lab script for what the constants $\gamma, \omega_0, f$ represent).

As we've done before, let's write this as a system of two first-order differential equations:

\begin{align}
    \frac{dv}{dt}
        &= f - 2 \gamma \omega_0 v - \omega_0^2 x \\
    \frac{dx}{dt}
        &= v
    .
\end{align}

Let's write a function that returns these derivatives.

In [ ]:
def fn_dho(
    t: float, x: float, v: float, gamma: float, omega0: float, f: float
) -> list:
    """Derivatives for damped harmonic oscillator."""
    return [v, f - 2 * gamma * omega0 * v - omega0 ** 2 * x]

Now we will solve this system using `solve_ivp`, trying out different values of $\gamma$.

In [ ]:
# Times to evaluate at
h = 0.01
ts = np.arange(0, 10 + h, step=h)

t_span = (0, 10)

# Try with different values of gamma
gammas = [0, 0.25, 0.5, 1, 2]

# Initial conditions
ics = [0, 0]

# Other params
f_val = 1
omega0 = 2

In [ ]:
# Gather the results
results = [
    integrate.solve_ivp(
        fun=lambda t, y: fn_dho(
            t=t, x=y[0], v=y[1], gamma=gamma, omega0=omega0, f=f_val
        ),
        t_span=t_span,
        y0=ics,
        t_eval=ts,
    )
    for gamma in gammas
]

Let's plot our results.

In [ ]:
# Plot
_, ax = plt.subplots()

for res in results:
    plt.plot(res.t, res.y[0])

# Labels and legend
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"$x$")

ax.legend([r"$\gamma$ = %.2f" % gamma for gamma in gammas]);

For the $\gamma = 0$, obviously there is no damping. Let's have a look at the non-zero values of $\gamma$ in a separate plot.

In [ ]:
# Plot
_, ax = plt.subplots()

# This plot is going to be crowded so let's use thinner lines
line_width = 0.7

# Plot dotted line at x = 0.25
plt.plot(np.linspace(0, 10, 250), 0.25 * np.ones(250), "--", lw=line_width)

# Print the non-zero gamma results
for res in results[1:]:
    plt.plot(res.t, res.y[0], lw=line_width)

# Labels and legend
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"$x$")

ax.legend(["x = 0.25"] + [r"$\gamma$ = %.2f" % gamma for gamma in gammas[1:]]);

The line $x = 0.25$ is plotted in blue, which makes it easier to see the behaviour of our results. We see that for $\gamma < 1$ our solutions are underdamped, the solution for $\gamma = 1$ turns out to be critically damped, and for $\gamma > 1$ our solutions are overdamped.

# Session 2 homework

Now we'll look at a similar system as we saw before, except this time with time dependent forcing:

\begin{equation}
    \frac{d^2 x}{dt^2} + 2 \gamma \omega \frac{dx}{dt} + \omega^2 x = f \cos \omega t
    .
\end{equation}

We can write this as the following system of ODEs:

\begin{align}
    \frac{dv}{dt}
        &= f \cos \omega t - 2 \gamma \omega v - \omega^2 x \\
    \frac{dx}{dt}
        &= v
    .
\end{align}

Let's define a function for this.

In [ ]:
def fn_dho_forced(
    t: float, x: float, v: float, gamma: float, omega: float, f: float
) -> list:
    """Derivatives for damped harmonic oscillator."""
    return [v, f * np.cos(omega * t) - 2 * gamma * omega * v - omega0 ** 2 * x]

## 1. Varying $\omega$

Continuing from the application we just looked at, let's fix $\gamma = 0.2$ (so we will have underdamped behaviour). Let's instead vary $\omega$.

In [ ]:
# Times to evaluate at
start_t = 0
end_t = 30

ts = np.linspace(start_t, end_t, 1000)

# Try with different values of omega
omega_0 = 2
omega_scaling_factors = [0.5, 0.95, 1.45]

omegas = omega_0 * np.array(omega_scaling_factors)

# Initial conditions
ics = [0, 0] 

# Other params
f_val = 1
gamma = 0.2

In [ ]:
# Gather the results
results = [
    integrate.solve_ivp(
        fun=lambda t, y: fn_dho_forced(
            t=t, x=y[0], v=y[1], gamma=gamma, omega=omega, f=f_val
        ),
        t_span=(start_t, end_t),
        y0=ics,
        t_eval=ts,
    )
    for omega in omegas
]

In [ ]:
# Plot
_, ax = plt.subplots()

for res in results:
    plt.plot(res.t, res.y[0])

# Labels and legend
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"$x$")

ax.legend([r"$\omega$ = %.2f $\omega_0$" % sf for sf in omega_scaling_factors]);

Let's first note that the amplitudes are much greater for $0.95 \omega_0$ than the others. The other difference is that as we increase $\omega$, the frequency of oscillations gets higher.

## 2. Damping ratios

Now, looking at $\gamma = 0.2, 0.5$, we will plot the amplitude of oscillations for different frequencies $\omega$.

In [ ]:
# Gamma values to consider
gammas = [0.2, 0.5]
num_gammas = len(gammas)

# Omegas to use
omega0 = 2
num_omegas = 500

omega_scaling_factors = np.linspace(0.2, 2, num_omegas)
omegas = omega0 * omega_scaling_factors

# Time range - make sure the maximum amplitude will occur in this range
start_t = 0
end_t = 20

# Initial conditions
ics = [0, 0]

# Other params
f_val = 1

In [ ]:
# For each gamma, find the amplitude for each omega
data = np.zeros((num_gammas, num_omegas))

for row, gamma in enumerate(gammas):
    for col, omega in enumerate(omegas):
        data[row][col] = (
            integrate.solve_ivp(
                fun=lambda t, y: fn_dho_forced(
                    t=t, x=y[0], v=y[1], gamma=gamma, omega=omega, f=f_val
                ),
                t_span=(start_t, end_t),
                y0=ics,
            )
            .y[0]
            .max()
        )

Now let's plot the results.

In [ ]:
# Create figure and axes
fig, axes = plt.subplots(num_gammas, 1)
fig.set_size_inches(10, 15)

# Add titles
for idx, ax in enumerate(axes):
    ax.title.set_text(r"$\gamma = %.2f$" % gammas[idx])

# Add labels
for ax in axes:
    ax.set_xlabel(r"$\omega_0$")
    ax.set_ylabel("amplitude")

# Adjust limits
for ax in axes:
    ax.set_ylim(0, 1)

# Plot
for idx, ax in enumerate(axes):
    ax.plot(omega_scaling_factors, data[idx])
    ax.grid(alpha=0.4)

Here we see that in both cases the general shape of the plots is the same: they start off descending, then they have a "bump", then they decrease again (and, although it is not plotted, it appears as if they decay to zero). The differences in the two plots is mainly that in the case for the smaller $\gamma$, the bump is more pronounced, while it is much less pronounced for the larger $\gamma$ value.